In [4]:
pip install --upgrade tensorflow

     |████████████████████████████████| 165.2 MB 25.1 MB/s eta 0:00:01    |███████████████████▎            | 99.3 MB 840 kB/s eta 0:01:19
     |████████████████████████████████| 57 kB 9.8 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 3.4 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 10.8 MB/s eta 0:00:0110.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.5 MB/s  eta 0:00:01
     |████████████████████████████████| 114 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 779 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 6.8 MB/s eta 0:00:01
     |██████████████

In [ ]:
python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [22]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install pydicom

Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
import cv2 as cv
import numpy as np
import pydicom as PDCM

def Dicom_to_Image(Path):
    DCM_Img = PDCM.read_file(Path, force=True)

    rows = DCM_Img.get(0x00280010).value
    cols = DCM_Img.get(0x00280011).value 

    Instance_Number = int(DCM_Img.get(0x00200013).value) 

    Window_Center = int(DCM_Img.get(0x00281050).value) 
    Window_Width = int(DCM_Img.get(0x00281051).value) 

    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): 
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): 
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) 

    return New_Img, Instance_Number

def main():
    Input_Folder = '/Users/conueyn/Documents/Science Research/CT Scans/Predictions/HNSCC dicom/HNSCC-01-0010'
    Output_Folder = '/Users/conueyn/Documents/Science Research/CT Scans/HNSCC Prediction 10'

    Input_Image_List = os.listdir(Input_Folder)

    if os.path.isdir(Output_Folder) is False:
        os.mkdir(Output_Folder)
    
    for i in range(0, len(Input_Image_List)):

        Output_Image, Instance_Number = Dicom_to_Image(Input_Folder + '/' + Input_Image_List[i])

        cv.imwrite(Output_Folder + '/' + str(Instance_Number - 1).zfill(4) + '.jpg', Output_Image)

if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'value'